In [1]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.guillaumerozier.fr
Mail : guillaume.rozier@telecomnancy.net

README:s
This file contains script that generate France maps and GIFs. 
Single images are exported to folders in 'charts/image/france'. GIFs are exported to 'charts/image/france'.
I'm currently cleaning this file, please ask me is something is not clear enough!
Requirements: please see the imports below (use pip3 to install them).

"""

"\n\nLICENSE MIT\n2020\nGuillaume Rozier\nWebsite : http://www.guillaumerozier.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:s\nThis file contains script that generate France maps and GIFs. \nSingle images are exported to folders in 'charts/image/france'. GIFs are exported to 'charts/image/france'.\nI'm currently cleaning this file, please ask me is something is not clear enough!\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [11]:
import france_data_management as data
import pandas as pd
from tqdm import tqdm
import json
import plotly.express as px
from datetime import datetime
import imageio
import multiprocessing
import locale
import shutil
import subprocess
import os
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
PATH = "../../"

In [3]:
# Import data from Santé publique France
_, _, _, _, _, _, _, df_incid, _ = data.import_data()
df_incid = df_incid[df_incid["cl_age90"] == 0]

with open(PATH+'data/france/dep.geojson') as response:
    depa = json.load(response)

  0%|          | 0/8 [00:00<?, ?it/s]/Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.

36it [02:27,  4.55s/it]                      /Users/guillaumerozier/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning:

Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.



In [22]:
def build_map(data_df, img_folder, date_val, date_str = "date", dep_str = "departement", color_str = 'indic_synthese', legend_title="legend_title", title="title", subtitle="", subsubtitle="{}<br>{} (données du {})", color_descrete_map={"Risque Faible":"#DAF7A6", "Alerte":"#b8002a", "Alerte Renforcée":"#7c0030", "Alerte Maximale":"#460d37"}):
    for date in date_val:
        data_df_temp = data_df[data_df[date_str] == date]
        
        files = os.listdir(img_folder)
        if "{}.jpeg".format(date) in files:
            print("map already generated", (img_folder+"/{}.jpeg").format(date))
            continue
        
        if len(data_df_temp) > 0:
            fig = px.choropleth(geojson = depa, 
                                locations = data_df_temp[dep_str], 
                                featureidkey="properties.code",
                                color = data_df_temp[color_str],
                                scope='europe',
                                #labels={color_str:"Couleur"},
                                #color_discrete_sequence = ["green", "orange", "red"],
                                #labels={'red':"Couleur", 'orange':'bla', 'green':'lol'},
                                color_discrete_map = color_descrete_map
                                #color_discrete_map = ,
                                #category_orders = {color_str :["Risque Faible", "Alerte", "Alerte Renforcée", "Alerte Maximale"]}
                                      )
            date_title = datetime.strptime(date, '%Y-%m-%d').strftime('%d %B')
            date_now = datetime.now().strftime('%d %B')

            fig.update_geos(fitbounds="locations", visible=False)

            fig.update_layout(
                legend_title_text = "Couleur",
                margin={"r":0,"t":0,"l":0,"b":0},
                title={
                    'text': title,
                    'y':0.98,
                    'x':0.5,
                    'font': {'size': 30},
                    'xanchor': 'center',
                    'yanchor': 'top'},

                titlefont = dict(
                    size=30),

                annotations = [
                    dict(
                        x=0.54,
                        y=0.03,
                        xref='paper',
                        yref='paper',
                        xanchor = 'center',
                        text='Source : Santé publique France. Auteur : @guillaumerozier - covidtracker.fr',
                        showarrow = False
                    ),

                    dict(
                        x=0.55,
                        y=0.94,
                        xref='paper',
                        yref='paper', 
                        text= subsubtitle.format(subtitle, date_now, date_title),
                        showarrow = False,
                        font=dict(
                            size=15
                                )
                    )]
                 ) 

            fig.update_geos(
                #center=dict(lon=-30, lat=-30),
                projection_rotation=dict(lon=12, lat=32, roll=8),
                #lataxis_range=[-50,20], lonaxis_range=[0, 200]
            )

            if date == dates_deconf[-1]:
                fig.write_image((img_folder+"/{}.jpeg").format("latest"), scale=2, width=1200, height=700)
            fig.write_image((img_folder+"/{}.jpeg").format(date), scale=1, width=1200, height=700)
        else:
            print("no data")

In [14]:
def build_gif(file_gif, imgs_folder, dates):
    i=0
    with imageio.get_writer(file_gif, mode='I', duration=0.3) as writer: 
        for date in tqdm(dates):
            try:
                print((imgs_folder+"/{}.jpeg").format(date))
                image = imageio.imread((imgs_folder+"/{}.jpeg").format(date))
                writer.append_data(image)
                i+=1
                if (i==len(dates)-1) or (i==0):
                    for k in range(8):
                        writer.append_data(image)
            except:
                print("no image for "+str(date))
    subprocess.run(["gifsicle", "-i", file_gif, "--optimize=1", "--scale=0.8", "--colors=180", "-o", file_gif[:-4]+"_opti.gif"])
    os.remove(file_gif)

In [24]:
dates_deconf = list(dict.fromkeys(list(df_incid["jour"].values)))

date = dates_deconf[-33:]
build_map(df_incid.sort_values(by=['incidence']), PATH+"images/charts/france/dep-map-incid-cat", date_val=date, date_str = "jour", dep_str = "dep", color_str = 'incidence_color', legend_title="", title="Incidence", subtitle="Nombre de cas hebdomadaires pour 100 000 habitants")

map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-21.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-22.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-23.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-24.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-25.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-26.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-28.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-29.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-06-30.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-07-01.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-07-02.jpeg
map already generated ../../images/charts/france/dep-map-incid-cat/2021-07-03.jpeg
map 

In [7]:
"""df_incid #df_incid.loc[:,"color_couvre_feu"] = 
deps_couvre_feu = ["01", "05", "06", "07", "08", "09", "10", "12", "13", "14", "67", "2A", "2B", "21", "26", "30", "31", "34", "35", "37", "38", "39", "42", "43", "45", "48", "49", "51", "54", "59", "60","62", "63", "64", "65", "66","67", "69", "71", "73","74", "75", "76", "77", "78", "81", "82", "83", "84", "87", "91", "92", "93", "94", "95"]
df_incid.loc[:,"color_couvre_feu"] = ['Couvre-feu' if dep in deps_couvre_feu else 'Pas de couvre-feu' for dep in df_incid['dep']]

dates_deconf = list(dict.fromkeys(list(df_incid["jour"].values)))
date = [dates_deconf[-1]]
build_map(df_incid.sort_values(by=['incidence']), "images/charts/france/dep-map-couvre-feu", date_val=date, date_str = "jour", dep_str = "dep", color_str = 'color_couvre_feu', legend_title="", title="Départements possiblement en couvre feu samedi", subsubtitle="", color_descrete_map={"Pas de couvre-feu":"#a4bda8", "Couvre-feu":"#bd2828"})"""

'df_incid #df_incid.loc[:,"color_couvre_feu"] = \ndeps_couvre_feu = ["01", "05", "06", "07", "08", "09", "10", "12", "13", "14", "67", "2A", "2B", "21", "26", "30", "31", "34", "35", "37", "38", "39", "42", "43", "45", "48", "49", "51", "54", "59", "60","62", "63", "64", "65", "66","67", "69", "71", "73","74", "75", "76", "77", "78", "81", "82", "83", "84", "87", "91", "92", "93", "94", "95"]\ndf_incid.loc[:,"color_couvre_feu"] = [\'Couvre-feu\' if dep in deps_couvre_feu else \'Pas de couvre-feu\' for dep in df_incid[\'dep\']]\n\ndates_deconf = list(dict.fromkeys(list(df_incid["jour"].values)))\ndate = [dates_deconf[-1]]\nbuild_map(df_incid.sort_values(by=[\'incidence\']), "images/charts/france/dep-map-couvre-feu", date_val=date, date_str = "jour", dep_str = "dep", color_str = \'color_couvre_feu\', legend_title="", title="Départements possiblement en couvre feu samedi", subsubtitle="", color_descrete_map={"Pas de couvre-feu":"#a4bda8", "Couvre-feu":"#bd2828"})'

In [8]:
"""deps_strings=[]
for dep in deps_couvre_feu:
    deps_strings += [df_incid[df_incid["dep"] == dep]["departmentName"].values[0]]
    
to_disp=""
for val in deps_strings:
    to_disp += val+", "
to_disp"""

'deps_strings=[]\nfor dep in deps_couvre_feu:\n    deps_strings += [df_incid[df_incid["dep"] == dep]["departmentName"].values[0]]\n    \nto_disp=""\nfor val in deps_strings:\n    to_disp += val+", "\nto_disp'

In [25]:
build_gif(PATH+"images/charts/france/incid-cat.gif", PATH+"images/charts/france/dep-map-incid-cat", dates_deconf[-33:])


  0%|          | 0/33 [00:00<?, ?it/s]

../../images/charts/france/dep-map-incid-cat/2021-06-21.jpeg



  3%|▎         | 1/33 [00:00<00:16,  1.93it/s]

../../images/charts/france/dep-map-incid-cat/2021-06-22.jpeg



  6%|▌         | 2/33 [00:01<00:17,  1.72it/s]

...


 94%|█████████▍| 31/33 [00:11<00:00,  2.90it/s]

../../images/charts/france/dep-map-incid-cat/2021-07-22.jpeg



 97%|█████████▋| 32/33 [00:13<00:00,  1.01it/s]

../../images/charts/france/dep-map-incid-cat/2021-07-23.jpeg



100%|██████████| 33/33 [00:14<00:00,  2.31it/s]
